###  0.1 Import Libraries

In [1]:
from exchange_getter import ExchangeGetter
from exchange_engine import ExchangeEngine

### 0.2 Create Global Variables (Likely taken from a config file)

In [2]:
currency_exchange_list = ['USDCAD', 'GBPCAD', 'EURCAD']
file_path = r"C:\Users\rmu\Documents\GitHub\CA-Risk\Bloomberg FX Fwd Curve.xlsx"
start_month = 1
start_year =  2024

### 1 Load Currency Exchange Rates

In [3]:
currency_exchange_dict = ExchangeGetter.get_currency_exchange(file_path=file_path, 
                                                              currency_exchange_list=currency_exchange_list)

### 2 Define Currency Data Pipeline

In [4]:
# define currency exchange cleaning processes
exchange_engine = ExchangeEngine(start_month = start_month, start_year = start_year)

process_pipeline = [exchange_engine.clean_currency_exchange_dfs, # select columns, drop empty cells, etc.
                    exchange_engine.add_year_month, # transform date, add year and month columns
                    exchange_engine.calculate_mean_exchange_rate, # calculate mean exchange based on forward bid and ask
                    exchange_engine.select_starting_point_rate, # select the starting point rate for start year and month
                    exchange_engine.forward_fill_exchange_rate,# forward fill the exchange rate for the next 100 years
                    ]

### 3 Deploy Defined Process workflow to Currency Exchange Data

In [5]:
for currency_key in currency_exchange_dict:
    for process in process_pipeline:
        currency_exchange_dict[currency_key] = process(currency_exchange_dict[currency_key])
    
    # update column 'mean_exchange_rate' base on dict key value
    currency_exchange_dict[currency_key] = exchange_engine.rename_df_based_on_key(currency_exchange_dict[currency_key], currency_key)

### 4 Combine All Exchange Rate in 1 mega dataframe ease for search

In [6]:
#combine all exchange rate into 1 dataframe, base on year and month column
mega_exchange_rate_df = exchange_engine.combine_currency_exchange_dfs(list(currency_exchange_dict.values()))

### 5 Load Mega Exchange Rate Data into Exchange Engine

In [7]:
exchange_engine.load_mega_exchange_rate(mega_exchange_rate_df)

# export exchange rate
exchange_engine.export_exchange_df('result.csv')

In [8]:
mega_exchange_rate_df

,year,month,USDCAD_Exchange_Rate,GBPCAD_Exchange_Rate,EURCAD_Exchange_Rate
0,2024,1,1.324300,1.687150,1.460610
1,2024,2,1.323711,1.686666,1.461825
2,2024,3,1.323165,1.686259,1.462903
3,2024,4,1.322672,1.685818,1.464187
4,2024,5,1.322168,1.685236,1.465499
...,...,...,...,...,...
1207,2124,8,1.114919,1.521172,1.707241
1208,2124,9,1.114919,1.521172,1.707241
1209,2124,10,1.114919,1.521172,1.707241
1210,2124,11,1.114919,1.521172,1.707241
